In [ ]:
!pip install tweepy==4.12.0
import tweepy as tw
import pandas as pd
import requests
import configparser
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Generate these details from Twitter Developer account!

In [ ]:
# These details are generated from twitter developer available at:https://developer.twitter.com/en/portal/dashboard
accesstoken = "accesstoken"
accesstokensecret = "accesstokensecret"
apikey = "apikey"
apisecretkey = "apisecretkey"
bearertoken = "bearertoken"

Creating functions for retrieving the author details from the tweet API

In [ ]:
# config = configparser.ConfigParser(interpolation=None)
# config.read("twitter_api.conf")

# Authenticate to access twitter api
auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)
api = tw.API(auth)

client = tw.Client(bearer_token=bearertoken)

# Creating the author URL
def create_url(author_id):
    return "https://api.twitter.com/2/users/{}?user.fields=public_metrics".format(author_id)


# Specifying the required parameters
def get_params():
    return {"tweet.fields": "created_at"}

# Creating the headers
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


# Connecting to the client endpoint
def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def getAuthorData(author_id):
    bearer_token = bearertoken
    url = create_url(author_id)
    headers = create_headers(bearer_token)
    params = get_params()
    json_response = connect_to_endpoint(url, headers, params)
    return (json_response['data'])

Reading the author details from the data set of tweets belonging to the twenty most popular tweet users in the year 2017.

In [ ]:
df = pd.read_csv('tweets_11042022_L7D.csv', dtype={'AUTHOR': str})
res = []

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,4,6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Querying the author details using the author ID and the twitter API

In [ ]:
too_many_req = 'Request returned an error: 429 {"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}'

api_calls = 0
# Specifying the limit for api calls
authors = 3600
for author_id in df.AUTHOR.unique()[3882:4800]:
  print(api_calls,author_id)
    # print(api_calls, str(author_id), type(author_id))
  try:
      author_details = getAuthorData(author_id)
  except Exception as e:
      print(f"exception as API call {api_calls} for author {author_id}")
      print(e)
      if e == too_many_req:
          tdf = pd.DataFrame(res, columns=['AUTHOR', 'AUTHOR_NAME', 'AUTHOR_USERNAME', 'FOLLOWERS_COUNT',
                                                'FOLLOWING_COUNT', 'TWEET_COUNT', 'LISTED_COUNT'])
          tdf.to_csv(f'author_details_{authors}_to_{authors+api_calls}.csv', index=0)
          time.sleep(15*60)
          api_calls = 0
          continue
 #requesting the necessary attributes from the result object
  res.append([author_id, author_details['name'], author_details['username'], author_details['public_metrics']['followers_count'],
              author_details['public_metrics']['following_count'], author_details['public_metrics']['tweet_count'],
              author_details['public_metrics']['listed_count']])
  api_calls += 1
#     print(api_calls)
  if api_calls==299:
    #converting author details to a dataframe
      tdf = pd.DataFrame(res, columns=['AUTHOR', 'AUTHOR_NAME', 'AUTHOR_USERNAME', 'FOLLOWERS_COUNT',
                                            'FOLLOWING_COUNT', 'TWEET_COUNT', 'LISTED_COUNT'])
      #writing the dataframe into a csv file
      tdf.to_csv(
          f'author_details_{authors}_to_{authors+api_calls}.csv', index=0)
      authors += api_calls
      print(f"done for {authors}")
      print(f"sleep after {api_calls} calls")
      time.sleep(15*60)
      api_calls = 0


0 1343334659967623170
1 1287128386788298752
2 1354633109472468993
3 331124685
4 1458725568397074435
5 1326268574630031377
6 1246194259193102336
7 1213269039708946433
8 287369079
9 1558197732510154752
10 1004125728395481088
11 1262082343
12 931324784184508416
13 1910963935
14 1570760754026156033
15 795140650228293632
16 1105952364756959232
17 231462839
18 462534410
19 573593541
20 483904952
21 1564109242805927936
22 1586474675676119040
23 1442123510
24 1366876493637386242
25 1565248981202259968
26 1252471063730106368
27 1575271309893836802
28 894988385017790467
29 1566228883179212800
30 2324532110
31 1095881775677493248
32 14199907
33 198369693
34 860622085969891332
35 3171491112
36 1502350136684552201
37 1568948417673699330
38 628082962
39 1043307771284873226
40 1586526228340932614
41 1582695592567914498
42 803798574
43 1368384730459107330
44 1518757252714274816
45 1585852754681122817
exception as API call 45 for author 1585852754681122817
'data'
46 1586155351379951624
47 1348311872639

In [ ]:
# Writing the dataframe into the CSV file
tdf = pd.DataFrame(res, columns=['AUTHOR', 'AUTHOR_NAME', 'AUTHOR_USERNAME', 'FOLLOWERS_COUNT',
                                      'FOLLOWING_COUNT', 'TWEET_COUNT', 'LISTED_COUNT'])
tdf.to_csv(
    f'author_details_{authors}_to_{authors+api_calls}.csv', index=0)
authors += api_calls
print(f"done for {authors}")
print(f"sleep after {api_calls} calls")
time.sleep(15*60)
api_calls = 0


done for 3882
sleep after 282 calls


KeyboardInterrupt: ignored

In [ ]:
api_calls

282